In [4]:
#start here this initializes the required modules

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import ase
from ase import Atoms
from ase.io import read
from ase.build import molecule
from ase.io import gamess_us
from ase.io.gamess_us import read_gamess_us_out
from ase.visualize import view
from ase.build import molecule
from weas_widget import WeasWidget
from ase.calculators.gamess_us import GAMESSUS
from ase.io import write
import re

#now determine how many cores we have
ncor = ! nproc

slist_string = " ".join(ncor)
try:
    integer_value = int(slist_string)
    print("Converted value:", integer_value)
except ValueError:
    print("Unable to convert to integer:", slist_string)

ncor=integer_value

def clean_for_new_job(name):

    # clean up output name in current folder
    nameout= name + '.out'
    if os.path.exists(nameout):
        # If the file exists, delete it
        os.remove(nameout)
        print(f"{nameout} has been deleted.")

    #clean output in extra folders

    # Specify the directory path where the files are located
    directory_path = "/scratch/cmuhich/gamess/restart"

    # Specify the pattern of files you want to delete using wildcard characters
    pattern = name + ".*"  # Example pattern, you can modify it as needed

    # Combine the directory path and pattern to form the full pattern
    full_pattern = os.path.join(directory_path, pattern)

    # Get a list of file paths that match the full pattern
    file_paths = glob.glob(full_pattern)

    # Check if there are any files matching the pattern
    if file_paths:
        # Iterate through the list of file paths
        for file_path in file_paths:
            # Delete each file
            os.remove(file_path)
            print(f"{file_path} has been deleted.")
    else:
        print(f"No files matching the pattern {pattern} were found in {directory_path}.")

    directory_path = "/home/cmuhich/gamess_scr"

    # Combine the directory path and pattern to form the full pattern
    full_pattern = os.path.join(directory_path, pattern)

    # Get a list of file paths that match the full pattern
    file_paths = glob.glob(full_pattern)

    # Check if there are any files matching the pattern
    if file_paths:
        # Iterate through the list of file paths
        for file_path in file_paths:
            # Delete each file
            os.remove(file_path)
            print(f"{file_path} has been deleted.")
    else:
        print(f"No files matching the pattern {pattern} were found in {directory_path}.")


def get_energy(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    energy_pattern = re.compile(r"TOTAL ENERGY =\s+([-+]?\d*\.\d+|\d+)")
    energies = []
    for line in lines:
        match = energy_pattern.search(line)
        if match:
            energy = float(match.group(1))
            energies.append(energy)
    
    
    if energies:
        print("Final Energies:")
        for i, energy in enumerate(energies, start=1):
            print(f"Run {i}: {energy} Hartree")
    else:
        print("No energy information found in the output file.")
    
    
    return energies

Converted value: 4


In [27]:
# pull up GUI for design
symbols = ['H', 'H', 'H']
positions = [[0, 0, 0], [0, 0, 0.98], [0, 0, 1.9]]

atoms= Atoms(symbols=symbols, positions=positions)


viewer = WeasWidget()
viewer.from_ase(atoms)
viewer

#atom manipulation:
# click atom or group (using the shift key)
# to move press "g" then do the motion
#to rotate press "r" then do the motion
# to duplicate press "d" 
# to delete press the delete key

WeasWidget(atomScales=[1, 1, 1], atoms={'species': {'H': ['H', 1]}, 'cell': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0…

In [28]:
# store information
atoms=viewer.to_ase()


In [29]:
#make calcuation choices
calc = GAMESSUS(contrl=dict(scftyp='rohf', dfttyp='blyp', runtyp='Hessian', mult=2),
                basis=dict(gbasis='n31', ngauss=6, ndfunc=1, npfunc=1),
                force=dict( METHOD='SEMINUM') )


#select name and run job

name='H3hessian'
namein= name + '.inp'
nameout= name + '.out'


atoms.set_calculator(calc)
calc.write_input( atoms)
! mv gamess_us.inp $namein

clean_for_new_job(name)

! ~/bin/rungms_basic $namein 00 $ncor > $nameout

H3hessian.out has been deleted.
No files matching the pattern H3hessian.* were found in /scratch/cmuhich/gamess/restart.
/home/cmuhich/gamess_scr/H3hessian.rst has been deleted.
/home/cmuhich/gamess_scr/H3hessian.dat has been deleted.
cp ./H3hessian.inp /scratch/cmuhich/gamess_scr/H3hessian.F05
unset echo
/home/onwokonk/gamess_basic/ddikick.x /home/onwokonk/gamess_basic/gamess.00.x H3hessian -ddi 1 4 c008.sol.rc.asu.edu:cpus=4 -scr /scratch/cmuhich/gamess_scr
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
unset echo
ls: No match.
ls: No match.
ls: No match.


In [30]:
#this will tell you the frequencies in cm-1
! grep 'FREQUENCY' $nameout

!grep 'IR INTENSITY' $nameout

       FREQUENCY:      1141.94 I      0.02        0.02        0.16      235.11  
       FREQUENCY:       235.11     1042.58     1042.58     2023.18  
   1 IMAGINARY FREQUENCY VIBRATION(S) IS(ARE) EXCLUDED
    IR INTENSITY:      1.39401     0.00000     0.00000     0.00000     0.00035
    IR INTENSITY:      0.00035     0.02121     0.02121     0.09653


In [33]:
#restart calcuation as a transtion search

calc = GAMESSUS(contrl=dict(scftyp='rohf', dfttyp='blyp', runtyp='SADPOINT', mult=2),
                basis=dict(gbasis='n31', ngauss=6, ndfunc=1, npfunc=1),
                force=dict( METHOD='SEMINUM'),
                statpt=dict(nstep=20, hess='calc', HSSEND='.TRUE.'))


#select name and run job

name='H3_tst'
namein= name + '.inp'
nameout= name + '.out'


atoms.set_calculator(calc)
calc.write_input( atoms)
! mv gamess_us.inp $namein

clean_for_new_job(name)

! ~/bin/rungms_basic $namein 00 $ncor > $nameout

H3_tst.out has been deleted.
No files matching the pattern H3_tst.* were found in /scratch/cmuhich/gamess/restart.
/home/cmuhich/gamess_scr/H3_tst.rst has been deleted.
/home/cmuhich/gamess_scr/H3_tst.dat has been deleted.
cp ./H3_tst.inp /scratch/cmuhich/gamess_scr/H3_tst.F05
unset echo
/home/onwokonk/gamess_basic/ddikick.x /home/onwokonk/gamess_basic/gamess.00.x H3_tst -ddi 1 4 c008.sol.rc.asu.edu:cpus=4 -scr /scratch/cmuhich/gamess_scr
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
unset echo
ls: No match.
ls: No match.
ls: No match.


In [35]:
atom = ase.io.read(nameout)
viewer.from_ase(atom)
viewer

WeasWidget(atomScales=[1, 1, 1], atoms={'species': {'H': ['H', 1]}, 'cell': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0…

In [34]:
#this will tell you the frequencies in cm-1
! grep 'FREQUENCY' $nameout

! grep 'IR INTENSITY' $nameout

! grep -C 10 'E         H         G         CV        CP        S' $nameout

       FREQUENCY:       819.06 I     26.65       26.65        0.01        0.01  
       FREQUENCY:         0.46      987.47      987.47     2059.95  
   1 IMAGINARY FREQUENCY VIBRATION(S) IS(ARE) EXCLUDED
    IR INTENSITY:      1.78258     0.00000     0.00000     0.00000     0.00000
    IR INTENSITY:      0.00000     0.02206     0.02206     0.00003
        0.009192 HARTREE/MOLECULE     2017.446020 CM**-1/MOLECULE 
        5.768173 KCAL/MOL               24.134038 KJ/MOL

               Q               LN Q
 ELEC.     2.00000E+00       0.693147
 TRANS.    2.06639E+05      12.238731
 ROT.      1.08277E+01       2.382104
 VIB.      1.01731E+00       0.017162
 TOT.      4.55231E+06      15.331145

              E         H         G         CV        CP        S
           KJ/MOL    KJ/MOL    KJ/MOL   J/MOL-K   J/MOL-K   J/MOL-K
 ELEC.      0.000     0.000    -1.718     0.000     0.000     5.763
 TRANS.     3.718     6.197   -30.339    12.472    20.786   122.544
 ROT.       2.479     2.479